In [1]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [2]:
!pip install unidecode
!pip install bpe
%cd /content/drive/MyDrive/NLU_assignment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 8.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 389 kB 7.4 MB/s 
     |████████████████████████████████| 15.0 MB 63.6 MB/s 
     |████████████████████████████████| 843 kB 71.6 MB/s 
/content/drive/MyDrive/NLU_assignment


# Import Libraries

In [3]:
from dataLoader.utils import load_file, save_file
from dataLoader.utils import get_poems_and_poets
from tokenization.bpe_tokenization import BPE_Tokenization
from dataLoader.generator import DataGenerator
from models.gru_encoder_decoder import get_encoder_decoder
import pickle
from sklearn.model_selection import train_test_split
from inference import generate_batch
from evaluation import get_bleu_scores
from evaluation import find_rhyme_accuracy
from dataLoader.utils import split_context_out
from inference import inference
from inference import generate_new_sample
import pandas as pd
from inference import predict_and_save

# Loading dataset

In [4]:
'''poem_list, poet_list  = get_poems_and_poets(path = "dataset/ganjoor.csv",
                                            cleaning = True,
                                            constrained_poets = ['حافظ','فردوسی'])

poem_list_train, poem_list_test, poet_list_train, poet_list_test = train_test_split(poem_list, poet_list, test_size=0.3, shuffle=False)

save_file(poem_list_train, 'dataset/poem_list_train.pickle')
save_file(poem_list_test, 'dataset/poem_list_test.pickle')
save_file(poet_list_train, 'dataset/poet_list_train.pickle')
save_file(poet_list_test, 'dataset/poet_list_test.pickle')'''

poem_list_train = load_file('dataset/poem_list_train.pickle')
poem_list_test = load_file('dataset/poem_list_test.pickle')
poet_list_train = load_file('dataset/poet_list_train.pickle')
poet_list_test = load_file('dataset/poet_list_test.pickle')

# loading Byte Pair Encoding Tokenizer

In [5]:
# loading Byte Pair Encoding Tokenizer
bpet = load_file('tokenization/bpe_tokenization.pkl')

In [6]:
example = ' '.join(poem_list_train[300])
print(bpet.to_index(example))
print(bpet.to_sentence(bpet.to_index(example)) )

[23391, 7380, 3061, 13985, 24172, 9435, 2871, 11386, 5338, 19649, 3997, 15208, 23710, 11360, 25684, 9435, 10888, 2326, 8910, 10677, 12432, 25884, 23010, 14606, 23010, 26618, 5404, 14212, 22556, 10436, 1126, 6177, 9096, 10620, 1126, 500, 25393, 9115, 1126, 7371, 9329, 5338, 26957, 24221, 23010, 11456, 24405, 24757, 10677, 457, 12432, 23010, 14606, 10888, 11386, 12028, 22556, 26615, 24007, 23010, 25298, 26746, 5882, 2871, 2530, 13091, 22610, 5454, 12028, 15055, 23386, 26618, 20394, 12028, 11562, 23010, 12028, 24070, 14761, 12028, 7185, 23386, 26052, 11781, 24737, 26334, 2871, 14606, 23010, 12432, 11386, 25684, 24394, 3107, 22826, 13985, 25684, 25365, 9659, 7673, 2795, 14723, 21095, 21769, 1126, 19849, 24361, 22627, 14480, 22726, 2312, 12393, 18669, 23837, 11048, 27216, 20639, 5287, 21189, 22744, 12028, 9435, 23386, 9056, 23010, 146, 22726, 1126, 7645, 2530, 2318, 11277, 23391, 25709, 5338, 1364, 2312, 563, 23839, 22726, 2454, 15208, 2312, 403, 12028, 7371, 1126, 551, 25684, 403, 18615, 2

# Attention based gru encoder-decoder

In [7]:
from models.gru_encoder_decoder import Gru_encoder_decoder

model = Gru_encoder_decoder(len(bpet.bpe2idx), len(bpet.bpe2idx),1024, use_att = True, dropout_rate = 0.3)
model.compile(optimizer="rmsprop", loss='categorical_crossentropy', metrics = ["accuracy"])
model.summary()
model.load_weights('/content/drive/MyDrive/NLU_assignment/saved_model/attention_gru_encoder_decoder_model_weights5-hafez-ferdosi.h5')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 encoder_embedding (Embedding)  (None, None, 1024)   27902976    ['encoder_input[0][0]']          
                                                                                                  
 gru_encoder_layer (Gru_encoder  ((None, None, 1024)  6297600    ['encoder_embedding[0][0]']      
 )                              , (None, 1024))                                                   
                                                                                                  
 decoder_input (InputLayer)     [(None, None)]       0           []                           

# Separating encoder and decoder model

In [8]:
encoder_model, decoder_model = get_encoder_decoder(model, latent_dim = 1024, decoder_dim = len(bpet.bpe2idx), use_att = True)

# evaluation of traning set - context_size 1

In [ ]:
import pandas as pd
contexts, Y_out = split_context_out(poem_list_train,
                                    num_contexts = 1,
                                    max_num_generated = 8)

'''gen = generate_batch(contexts,
                     Y_out,
                     encoder_model, 
                     decoder_model, 
                     bpet, max_num_he = 8, 
                     save_path = "/content/drive/MyDrive/NLU_assignment/results/train/generation/1-8.csv")'''


df = pd.read_csv('/content/drive/MyDrive/NLU_assignment/results/train/generation/1-8.csv')

gen = df['hypotheses']
gen = list(gen)
gen = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in gen]

contexts = df['context']
contexts = list(contexts)
contexts = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in contexts]

Y_out = df['references']
Y_out = list(Y_out)
Y_out = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in Y_out]

print(get_bleu_scores(gen, Y_out, blue_max_n_gram = 2))
print(get_bleu_scores(gen, Y_out, blue_max_n_gram = 4))

print([find_rhyme_accuracy(contexts, [x[:len(x)-i] for x in gen]) for i in [5,3,1]])

([0.04210526315789474, 0.025000000000000005, 0.031413612565445025, 0.03888888888888889, 0.07027027027027027, 0.07582938388625593, 0.09302325581395349], 0.1166217235210703)
([0.024691358024691357, 0.013513513513513514, 0.016574585635359115, 0.020348837209302327, 0.036312849162011177, 0.03902439024390244, 0.047619047619047616], 0.07675774879855557)
[0.7298226061112659, 0.6862470862470863, 0.6576148659607498]


# evaluation of traning set - context_size 2

In [ ]:
contexts, Y_out = split_context_out(poem_list_train,
                                    num_contexts = 2,
                                    max_num_generated = 8)

'''gen = generate_batch(contexts,
                     Y_out,
                     encoder_model, 
                     decoder_model, 
                     bpet, max_num_he = 8, 
                     save_path = "/content/drive/MyDrive/NLU_assignment/results/train/generation/2-8.csv")'''


df = pd.read_csv('/content/drive/MyDrive/NLU_assignment/results/train/generation/2-8.csv')

gen = df['hypotheses']
gen = list(gen)
gen = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in gen]

contexts = df['context']
contexts = list(contexts)
contexts = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in contexts]

Y_out = df['references']
Y_out = list(Y_out)
Y_out = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in Y_out]

print(get_bleu_scores(gen, Y_out, blue_max_n_gram = 2))
print(get_bleu_scores(gen, Y_out, blue_max_n_gram = 4))

print([find_rhyme_accuracy(contexts, [x[:len(x)-i] for x in gen]) for i in [6,4,2]])

([0.13186813186813187, 0.1513157894736842, 0.16666666666666666, 0.19591836734693877, 0.1984126984126984, 0.21076233183856502, 0.27325581395348836], 0.11878504599449088)
([0.11038961038961038, 0.12857142857142856, 0.14285714285714285, 0.17659574468085107, 0.17418032786885246, 0.1889400921658986, 0.24702380952380953], 0.07882656231832925)
[0.8211123408965135, 0.73430334361711, 0.6930367675107838]


# evaluation of traning set - context_size 4

In [ ]:
import pandas as pd

contexts, Y_out = split_context_out(poem_list_train,
                                    num_contexts = 4,
                                    max_num_generated = 8)

'''gen = generate_batch(contexts,
                     Y_out,
                     encoder_model, 
                     decoder_model, 
                     bpet, max_num_he = 8, 
                     save_path = "/content/drive/MyDrive/NLU_assignment/results/train/generation/4-8.csv")'''


df = pd.read_csv('/content/drive/MyDrive/NLU_assignment/results/train/generation/4-8.csv')

gen = df['hypotheses']
gen = list(gen)
gen = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in gen]

contexts = df['context']
contexts = list(contexts)
contexts = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in contexts]

Y_out = df['references']
Y_out = list(Y_out)
Y_out = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in Y_out]

print(get_bleu_scores(gen, Y_out, blue_max_n_gram = 2))
print(get_bleu_scores(gen, Y_out, blue_max_n_gram = 4))

print([find_rhyme_accuracy(contexts, [x[:len(x)-i] for x in gen]) for i in [6,4,2]])

([0.10112359550561799, 0.08433734939759036, 0.10382513661202186, 0.09345794392523364, 0.1073170731707317, 0.13924050632911392, 0.12941176470588237], 0.10386496138846807)
([0.09333333333333335, 0.06168831168831169, 0.06936416184971098, 0.06067961165048544, 0.0678391959798995, 0.08766233766233766, 0.08132530120481928], 0.06555420517969644)
[0.8300665567564267, 0.759965409323011, 0.720152817574021]


# evaluation of traning set - context_size 8

In [ ]:
contexts, Y_out = split_context_out(poem_list_train,
                                    num_contexts = 8,
                                    max_num_generated = 8)

'''gen = generate_batch(contexts,
                     Y_out,
                     encoder_model, 
                     decoder_model, 
                     bpet, max_num_he = 8, 
                     save_path = "/content/drive/MyDrive/NLU_assignment/results/train/generation/8-8.csv")'''

df = pd.read_csv('/content/drive/MyDrive/NLU_assignment/results/train/generation/8-8.csv')

gen = df['hypotheses']
gen = list(gen)
gen = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in gen]

contexts = df['context']
contexts = list(contexts)
contexts = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in contexts]

Y_out = df['references']
Y_out = list(Y_out)
Y_out = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in Y_out]

print(get_bleu_scores(gen, Y_out, blue_max_n_gram = 2))
print(get_bleu_scores(gen, Y_out, blue_max_n_gram = 4))

print([find_rhyme_accuracy(contexts, [x[:len(x)-i] for x in gen]) for i in [6,4,2]])

([0.15, 0.16939890710382513, 0.17567567567567569, 0.17427385892116182, 0.16942148760330578, 0.0684931506849315, 0.06896551724137931], 0.14651365771102862)
([0.13768115942028983, 0.1553030303030303, 0.16049382716049382, 0.1553672316384181, 0.15126050420168066, 0.046296296296296294, 0.046511627906976744], 0.1196426321118339)
[0.8706017757316672, 0.8032181810641562, 0.7581853128313892]


# evaluation of test set - context_size 1

In [ ]:
contexts, Y_out = split_context_out(poem_list_test,
                                    num_contexts = 1,
                                    max_num_generated = 8)

'''gen = generate_batch(contexts,
                     Y_out,
                     encoder_model, 
                     decoder_model, 
                     bpet, max_num_he = 8, 
                     save_path = "/content/drive/MyDrive/NLU_assignment/results/test/generation/1-8.csv")'''



df = pd.read_csv('/content/drive/MyDrive/NLU_assignment/results/test/generation/1-8.csv')

gen = df['hypotheses']
gen = list(gen)
gen = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in gen]

contexts = df['context']
contexts = list(contexts)
contexts = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in contexts]

Y_out = df['references']
Y_out = list(Y_out)
Y_out = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in Y_out]

print(get_bleu_scores(gen, Y_out, blue_max_n_gram = 2))
#print(get_bleu_scores(gen, Y_out, blue_max_n_gram = 4))

print([find_rhyme_accuracy(contexts, [x[:len(x)-i] for x in gen]) for i in [5,3,1]])

([0.019801980198019802, 0.04895104895104895, 0.05625, 0.04697986577181208, 0.043243243243243246, 0.07333333333333333, 0.08139534883720931], 0.07733586326509435)
[0.5034046692607004, 0.46785225718194257, 0.43973862536302033]


# evaluation of test set - context_size 2

In [ ]:
contexts, Y_out = split_context_out(poem_list_test,
                                    num_contexts = 2,
                                    max_num_generated = 8)

'''gen = generate_batch(contexts,
                     Y_out,
                     encoder_model, 
                     decoder_model, 
                     bpet, max_num_he = 8, 
                     save_path = "/content/drive/MyDrive/NLU_assignment/results/test/generation/2-8.csv")'''



df = pd.read_csv('/content/drive/MyDrive/NLU_assignment/results/test/generation/2-8.csv')

gen = df['hypotheses']
gen = list(gen)
gen = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in gen]

contexts = df['context']
contexts = list(contexts)
contexts = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in contexts]

Y_out = df['references']
Y_out = list(Y_out)
Y_out = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in Y_out]

print(get_bleu_scores(gen, Y_out, blue_max_n_gram = 2))
print(get_bleu_scores(gen, Y_out, blue_max_n_gram = 4))

print([find_rhyme_accuracy(contexts, [x[:len(x)-i] for x in gen]) for i in [6,4,2]])

([0.017391304347826087, 0.03684210526315789, 0.04564315352697095, 0.04583333333333333, 0.06666666666666667, 0.060240963855421686, 0.06097560975609756], 0.07647834892143764)
([0.009900990099009901, 0.019662921348314606, 0.023809523809523808, 0.023706896551724137, 0.03424657534246575, 0.030864197530864196, 0.03125], 0.03903501639119754)
[0.5438745231030098, 0.493167701863354, 0.45252492237293673]


# evaluation of test set - context_size 4

In [ ]:
contexts, Y_out = split_context_out(poem_list_test,
                                    num_contexts = 4,
                                    max_num_generated = 8)

'''gen = generate_batch(contexts,
                     Y_out,
                     encoder_model, 
                     decoder_model, 
                     bpet, max_num_he = 8, 
                     save_path = "/content/drive/MyDrive/NLU_assignment/results/test/generation/4-8.csv")'''



df = pd.read_csv('/content/drive/MyDrive/NLU_assignment/results/test/generation/4-8.csv')

gen = df['hypotheses']
gen = list(gen)
gen = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in gen]

contexts = df['context']
contexts = list(contexts)
contexts = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in contexts]

Y_out = df['references']
Y_out = list(Y_out)
Y_out = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in Y_out]

print(get_bleu_scores(gen, Y_out, blue_max_n_gram = 2))
print(get_bleu_scores(gen, Y_out, blue_max_n_gram = 4))

print([find_rhyme_accuracy(contexts, [x[:len(x)-i] for x in gen]) for i in [6,4,2]])

([0.008695652173913044, 0.05339805825242718, 0.044534412955465584, 0.030534351145038167, 0.046012269938650305, 0.03125, 0.03614457831325301], 0.0781046209565283)
([0.0049504950495049506, 0.028350515463917526, 0.023206751054852322, 0.015748031496062992, 0.02358490566037736, 0.015957446808510637, 0.018518518518518517], 0.03985821362334427)
[0.7128842380640942, 0.6077625570776256, 0.551286449399657]


# evaluation of test set - context_size 8

In [ ]:
contexts, Y_out = split_context_out(poem_list_test,
                                    num_contexts = 8,
                                    max_num_generated = 8)

'''gen = generate_batch(contexts,
                     Y_out,
                     encoder_model, 
                     decoder_model, 
                     bpet, max_num_he = 8, 
                     save_path = "/content/drive/MyDrive/NLU_assignment/results/test/generation/8-8.csv")'''



df = pd.read_csv('/content/drive/MyDrive/NLU_assignment/results/test/generation/8-8.csv')

gen = df['hypotheses']
gen = list(gen)
gen = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in gen]

contexts = df['context']
contexts = list(contexts)
contexts = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in contexts]

Y_out = df['references']
Y_out = list(Y_out)
Y_out = [[y.strip('\'') for y in x.strip('][').split(', ')] for x in Y_out]

print(get_bleu_scores(gen, Y_out, blue_max_n_gram = 2))
print(get_bleu_scores(gen, Y_out, blue_max_n_gram = 4))

print([find_rhyme_accuracy(contexts, [x[:len(x)-i] for x in gen]) for i in [6,4,2]])

([0.035211267605633804, 0.04149377593360996, 0.03642384105960265, 0.05472636815920398, 0.0684931506849315, 0.06896551724137931, 0.023529411764705882], 0.08020378249918174)
([0.019841269841269844, 0.022026431718061675, 0.01896551724137931, 0.028205128205128206, 0.035211267605633804, 0.03529411764705882, 0.012048192771084338], 0.04089165520735326)
[0.7661870503597122, 0.6651604157252599, 0.5932809149392423]


In [ ]:
predict_and_save(poem_list_test,
                 encoder_model, 
                 decoder_model, 
                 bpet,
                 path = '/content/drive/MyDrive/NLU_assignment/results/test/prediction/',
                 max_num_he = 16)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


# reading saved prediction

In [ ]:
df__ = pd.read_csv('/content/drive/MyDrive/NLU_assignment/results/train/prediction/prediction_he_6.csv')

references = list(df__['ground_truth'])
hypotheses = list(df__['predicted'])
 
references = [[x.split(' ')] for x in references]
hypotheses = [x.split(' ') for x in hypotheses]


for i in range(30):
    print('########################################')
    print('context:')
    print(df__.iloc[i]['context'])
    print('ground_truth:')
    print(df__.iloc[i]['ground_truth'])
    print('predicted:')
    print(df__.iloc[i]['predicted'])
    print('########################################')

########################################
context:
به نام خداوند جان و خرد 	 کزین برتر اندیشه برنگذرد 
 خداوند نام و خداوند جای 	 خداوند روزی ده رهنمای 
 خداوند کیوان و گردان سپهر
ground_truth:
فروزنده ماه و ناهید و مهر
predicted:
خداوند هستی و داد و مهر
########################################
########################################
context:
ز نام و نشان و گمان برترست 	 نگارنده بر شده پیکرست 
 به بینندگان آفریننده را 	 نبینی مرنجان دو بیننده را 
 نیابد بدو نیز اندیشه راه
ground_truth:
که او برتر از نام و از جایگاه
predicted:
که او برتر از نام و از جایگاه
########################################
########################################
context:
سخن هر چه زین گوهران بگذرد 	 نیابد بدو راه جان و خرد 
 خرد گر سخن برگزیند همی 	 همان را گزیند که بیند همی 
 ستودن نداند کس او را چو هست
ground_truth:
میان بندگی را ببایدت بست
predicted:
میان بندگی را ببایدت بست
########################################
########################################
context:
خرد را و جان را همی سنجد اوی 	 در اندیشه سخته

# Trying out some samples

In [9]:
input = 'نه وصل بماند و نه واصل'
out = inference(input,encoder_model, decoder_model, bpet, num_pads=5)

out

'که در دست باشد سپهر و زمین'

In [10]:
prefix = 'به دشمن کرد سپاهی روان'
print(generate_new_sample(prefix, encoder_model, decoder_model, bpet, num_he = 20, num_pads = 10))

به دشمن کرد سپاهی روان		بیامد به کردار باد روان
به پیش سپه شد چو دریای قیر		برفتند و بردند یک یک بجنگ
چو شد لشکر شهریار جوان		به گردان سپهر و به داد جوان
چنین گفت پس با سواران جنگ		که چون رزم باید شدن بی درنگ
سپهدارشان رستم شیر دل		که بر سنگ خارا شود دل گسل
نباید که چشم اندر آرد به شاه		نبیند چنو نیز یک تن سیاه
ورا هم بران گونه کارآگاه کن		سرش را ز تن دور کن گاه خواب
بدان تا مگر رنج افراسیاب		دلش گردد از درد و تیره آب
فرازآمد آن پیر گرگ سترگ		به بخت و به مردی همان تخت گرگ
به آب افگند ماهیان سی هزار		ز راه سواران خنجرگزار
برفتند هر دو به قلب سپاه		


In [ ]:
prefix = 'به دشمن کرد سپاهی سترگ'
print(generate_new_sample(prefix, encoder_model, decoder_model, bpet, num_he = 20, num_pads = 10))

به دشمن کرد سپاهی سترگ		فرستاد با پهلوانی بزرگ
به هر سو فرستاد کارآگهان		همی چست پیدا ز کار جهان
که بهرام جنگ آنک بد نامدار		برون تاختند از میان مهان
که تا زنده ام نام افراسیاب		مرا رفت با دل پر از درد و آب
بدان تا نهانی بود نام و ننگ		بیاورد پس با یکی کینه جنگ
همی گفت با من که افراسیاب		همی رفت با سیاوش بر سر
چنین گفت کای پهلوان سپاه		بزیبد من ایدر بباید شاه
همی خویشتن را به ایران سپاه		به خاک اندر آید سر تاج و گاه
نباشد شگفت ار به رنج و به درد		تن اژدها را به رنج نبرد
به گیتی درون دادمردی پدر		همی بود ازین پیش بهمن به زر
به خوبی سزای جهان داد و گفت		


In [ ]:
prefix = 'ز خرگاه لشکر به هامون کشید'
print(generate_new_sample(prefix, encoder_model, decoder_model, bpet, num_he = 20, num_pads = 10))

ز خرگاه لشکر به هامون کشید		سپه را سوی طیسفون کشید
به لشکر بفرمود پس شهریار		که برداشتند آلت کارزار
بدان آگهی نزد ایشان رسید		همی از هوا جوی خون شاخ دید
ز گرد سواران هوا تار شد		سپهدار کابل گرفتار شد
چنین گفت با نامداران گرد		که ما را خرددارماز چار پیل
چه کردم کزین بیچاری شاه را		مگر اختر نیک و بد لشکران را
کنون چون بجنگ اندر آید نخست		ره بازگشتن ببایدش جست
ازاینها که تو نام یزدان بود		همه کار بر پای و بر پای بود
بفرمود پس تا زبان برگشاد		ز پیش سپهبد مکن نیز یاد
که من شاه را دل پر از کین کنید		سپر بستر و تیغ بالین کنید
شوم تا کنم تیره خاک سیاه		


In [ ]:
prefix = 'ز خرگاه لشکر به هامون کشید\tهر آنچه داشت کشید'
print(generate_new_sample(prefix, encoder_model, decoder_model, bpet, num_he = 20, num_pads = 10))

ز خرگاه لشکر به هامون کشید		هر آنچه داشت کشید
به یک هفته لشکر بیاراستند		می و رود و رامشگران خواستند
چنین گفت پس با فرستاده مرد		که ایدر نه با رستم آرد به درد
اگر نه که با مهتر سرفراز		ز گردان ایران سپهشان به راز
ازان پس که بیم و زیانکار ماست		همان آرزوها که دیوارکرد
ز گیتی برآمد بسی گرم و سرد		ز گیتی کسی را شگفتی ندید
چو آتش ازان رزمگه بگذرد		دم رنجدم جان جان را سپرد
ازان پس گذرها که آمد پدید		ز لشکر به هر کشور آمد پدید
چو از رودهم بر پس اسپ و پیل		رده بر همی لب دو از دو میل
دو لشکر به اسپ و به چنگال دیو		همی جست خواهد همی از پور
اگر دور ماند به هم بازجوی		شود شاه ازین پس پر از گفت وگوی



In [ ]:
prefix = 'من سخن گفتن نتوانم\tاز لب تو چه سخن پیکر\nگر چه در سینه سنگ و گوهر بود'
print(generate_new_sample(prefix, encoder_model, decoder_model, bpet, num_he = 20, num_pads = 10))

من سخن گفتن نتوانم		از لب تو چه سخن پیکر
گر چه در سینه سنگ و گوهر بود		هست در پرهیزگاران  کبود
در سر گاه تو خورشید نیست		نه در خانه که پر جز دود نیست
نه در آب روان ترا کس نه		نه در گنج ز بهر جای ترا کس
نه آن را که نه در جهان آباد		نه در مردم و نه بوم ترا
چنین داد پاسخ که داننده مرد		که دانش ورا ننگ دارد به ننگ
دگر گفت مردم که ای شیرفش		به آب خرد باید آبکش ز خواب
مگر در یکی سنگ آتش کند		همان بر تنش بر سرش آتش کند
ازان پس که آمد به راه تو بست		همی بود تا پیش او شد پست
همی راند تا پیش او شد به کش		سخن را همی داشت هش دار کش
همی گفت با من که تن پیل کن		سپه را ز دشمن بدین بدتنه
ازان پس که شاه جهان پهلوان		


In [ ]:
prefix = 'نه وصل بماند و نه واصل'
print(generate_new_sample(prefix, encoder_model, decoder_model, bpet, num_he = 20, num_pads = 20))

نه وصل بماند و نه واصل		که در کردن آن رنج در دل کمند
نه آن کس که ماند در جهان		نه آن دید کز خویشتن در نهان
نه نیز از پس پرده بیرون کشید		نه هنگام تیمار و نه هامون کشید
همی راند منزل به منزل سپاه		ز کوه و بیابان وز آن کوه راه
بیابان که آن از در شارستان		بزیبد اندرون پیش خارستانستان
به سر بر نهادند یک روز بد		به چرخ اندرون بانگ برزد که دم
دگر روز چون گشت روشن روان		فرستاد باید به سوی پهلوان
مگر کز میان سپاه آمدی		همانا که با رای پیرانآمدی
به رزم اندرون رخش و رای و هنر		نژاد و بزرگی و فر و گهر
به پیش اندرون سام گیهان گشای		فرو هشته از تاج پر همای
ابا زال و با رای و با فر و برز		
